##### 01. Dynamic analysis

In [ ]:
import os

def make_only_file(directory, file_name, is_make_temp_dir = False):
    """파일명으로 생성"""
    if is_make_temp_dir is True :
        directory = mkdtemp()
    if len(directory) >= 2 and not os.path.exists(directory) :
        os.makedirs(directory)
    return os.path.join(file_name)

def make_path(directory, file_name, is_make_temp_dir = False):
    """디렉토리 + 파일명으로 경로 생성"""
    if is_make_temp_dir is True :
        directory = mkdtemp()
    if len(directory) >= 2 and not os.path.exists(directory) :
        os.makedirs(directory)
    return os.path.join(directory, file_name)

class Custom_Error(Exception) :
    pass

#%%
import pandas as pd
import re
import numpy as np
directory = 'C:/Users/chaesungwon/OneDrive - 서울대학교/csw/dtm_v4/input/patent/'
df = pd.read_csv(directory + 'drug_side_effect.csv',encoding='latin_1')
df['text'] = df['name'] + ' ' + df['Abstract']

df['YEAR'] = [str(i)[:4] for i in df['YEAR']]
docs = df["text"].tolist()


#from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
#from gensim.models import Phrases
from gensim.corpora import Dictionary
from nltk.corpus import wordnet
import nltk
import spacy

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''    
    
nlp = spacy.load("en_core_web_sm")

word_list = []

for idx in range(len(docs)):
    
    doc = nlp(docs[idx])
    
    temp = []
    for token in doc :
        temp.append(token.lemma_.lower())
    
    word_list.append(temp)  # word_list insert
    #word_list[idx] = word_list[idx].lower()  # Convert to lowercase.
    #word_list[idx] = tokenizer.tokenize(word_list[idx])  # Split into words.

rgxWord = re.compile('[a-zA-Z][-_a-zA-Z0-9.]*') 

# hoho
word_list = [[token for token in doc if rgxWord.match(token)] for doc in word_list]

# Remove words that are only one/two character.
word_list = [[token for token in doc if len(token) > 2] for doc in word_list]

is_noun = lambda pos: pos[:2] == 'NN'
# do the nlp stuff
word_list = [[token for (token, pos) in nltk.pos_tag(doc) if is_noun(pos)] for doc in word_list]

# 스탑워즈 제거

# stopwords_smart

f = open('C:/Users/chaesungwon/OneDrive - 서울대학교/csw/dtm_v4/input/stopwords_smart.txt', mode='rt', encoding='utf-8')
L = []

for line in f:
    L.append(line.strip())

stop = L

for i in stop:
    temp_word = i
    word_list = [[token for token in doc if not token == temp_word] for doc in word_list]
    
    
# stopwords_tech

f = open('C:/Users/chaesungwon/OneDrive - 서울대학교/csw/dtm_v4/input/stopwords_tech.txt', mode='rt', encoding='utf-8')
L = []

for line in f:
    L.append(line.strip())

stop = L

for i in stop:
    temp_word = i
    word_list = [[token for token in doc if not token == temp_word] for doc in word_list]


#%%

from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
from gensim.matutils import hellinger
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.wrappers import DtmModel
import numpy
import os
from datetime import datetime
from gensim.corpora import Dictionary

# path to dtm home folder
dtm_home = os.environ.get('DTM_HOME', "dtm-master")
# path to the binary. on my PC the executable file is dtm-master/bin/dtm
dtm_path = os.path.join(dtm_home, 'bin', 'dtm') if dtm_home else None
# you can also copy the path down directly. Change this variable to your DTM executable before running.
dtm_path = "C:/Users/chaesungwon/OneDrive - 서울대학교/csw/dtm_v4/input/dtm-master/bin/dtm-win64.exe"

#%%
#1년단위
try:
    time_count = df['YEAR'].value_counts().sort_index()
except:
    time_count = df['YEAR'].value_counts()

time_count = time_count.to_list()

dictionary = Dictionary(word_list)

corpus = [dictionary.doc2bow(doc) for doc in word_list]
num_topics = 20

dtm_model = DtmModel(dtm_path, corpus, time_count, num_topics=num_topics, id2word=dictionary,
                      initialize_lda=True, alpha=0.01, top_chain_var=0.05)

# dtm_model.save(DIR_output + 'dtm_model5')

DIR_output = 'C:/Users/chaesungwon/OneDrive - 서울대학교/csw/dtm_v4/output/'
#%% 문헌 토픽 비율 라벨링

topic_table = pd.DataFrame()

for i in range(len(docs)):
    temp = pd.Series(dtm_model.gamma_[i])
    topic_table = topic_table.append(temp, ignore_index=True)

topic_table.columns = ["topic_" + repr(x) for x in range(len(topic_table.iloc[0, :]))]
topic_df = pd.concat([topic_table, topic_table.idxmax(axis=1), topic_table.max(axis=1)], axis=1)
length = len(topic_table.iloc[0, :])

topic_df.columns.values[length] = "topic_num"
topic_df.columns.values[length + 1] = "max_prop"
topic_df['period'] = df['YEAR']

after_docs = docs[:]

for idx in range(len(docs)):
    after_docs[idx] = ' '.join(word_list[idx])
after_docs = pd.DataFrame(after_docs)

app_num = df.YEAR.tolist()
app_num = pd.DataFrame(app_num)
df_after = pd.concat([after_docs, topic_df.loc[:, "topic_num"], topic_df.loc[:, "max_prop"], app_num], axis=1)
df_after.columns = ["word_list", "main_topic", "topic_prop", "period"]
#df_after = df_after[df_after.topic_prop >= 0.30]
# df_after.to_csv(DIR_output + "doc_to_topic.csv", index=False)

topic_df = topic_df.drop(['topic_num','max_prop'], axis = 1)

import statistics

def make_word_vector(word):

    word = word
    temp_data = {'word': word}
    temp_set = list(set(df_after.loc[:, "period"]))
    temp_set = repr(temp_set)
    sorted(temp_set)

    count = 1

    for j in range(len(set(df_after.loc[:, "period"]))):  # 기간 동안의
        temp_period = temp_set[j]
        temp_data[repr(count)] = 0
        count += 1

    temp_data = pd.DataFrame([temp_data])
    temp_list = sorted(temp_data.iloc[:, 1:(len(temp_data.columns))], key= repr)
    temp_list.insert(0, 'word')

    temp_data = temp_data[temp_list]

    return (temp_data)

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error : creating directory." + directory)


def CAGR(x, last_col, first_col, num_periods):
    """ Calculate compound growth rate on a row x of a pandas df"""
    val_T = float(x[last_col]) + 0.00001
    val_t = float(x[first_col]) + 0.00001
    return ((val_T / val_t) ** (1. / num_periods) - 1) * 100
            
#%%

for i in range(num_topics):

    topic = i
    resultlist = []

    for j in range(len(set(df_after.loc[:, "period"]))):  # 기간 동안의
        temp = dtm_model.show_topic(i, j)
        resultlist.append(temp)

    df = pd.DataFrame(resultlist)

    dtw_df = make_word_vector('temp')

    for j in range(len(df)):

        for k in range(len(df.loc[j])):
            
            temp_word = df.loc[j][k][1]  # 단어 저장
            
            temp_list = dtw_df.loc[:, "word"].values.tolist()

            if temp_word in temp_list:
                dtw_df.loc[dtw_df.word == temp_word, repr(j + 1)] = df.loc[j][k][0]  # 확률저장
            else:
                temp_df = make_word_vector(temp_word)
                temp_df.iloc[:, j+1] = df.loc[j][k][0]  # 확률저장
                dtw_df = dtw_df.append(temp_df, ignore_index=True)

    dtw_df = dtw_df.iloc[1:, :]
    dtw_df.iloc[:, 1:].stack().idxmax()

    max_row = dtw_df.iloc[:, 1:].stack().idxmax()[0]  # 가장 큰 값의 row
    # max2_row = dtw_df.iloc[:,1:].stack().nlargest(2).index[1][0] # 2번째 큰 값의 row # 수정필요함

    if (i >= 10):
        DIR_temp = DIR_output + "topic_" + repr(i) + "_" + dtw_df.loc[max_row, "word"]
        createFolder(DIR_temp)
        DIR_temp = DIR_temp + "/" + "topic_" + repr(i) + "_" + dtw_df.loc[max_row, "word"]
        dtw_df.to_csv(DIR_temp + ".csv", header=False, index=False, encoding= "utf-8")
    else:
        DIR_temp = DIR_output + "topic_0" + repr(i) + "_" + dtw_df.loc[max_row, "word"]
        createFolder(DIR_temp)
        DIR_temp = DIR_temp + "/" + "topic_0" + repr(i) + "_" + dtw_df.loc[max_row, "word"]
        dtw_df.to_csv(DIR_temp + ".csv", header=False, index=False, encoding= "utf-8")


#%% 토픽 변화 시각화
import pandas as pd
import string as str
import numpy as np
import re
import os
import matplotlib.pyplot as plt


output_name_set = []

for f in [make_only_file(DIR_output, file) for file in os.listdir(path=DIR_output) if "topic_" in file]:
    output_name_set.append(f)

# temp_list1 = temp_df.sum(axis=0).sort_values(ascending=False)[0:5].index
# temp_df1 = temp_df.loc[:, temp_list1]
# temp_df1.plot(marker="v")

# 1. 상위 5개의 토픽 추출
topicSize_df = topic_df.groupby(topic_df['period']).sum()


temp_ = topicSize_df.sum(axis=0).sort_values(ascending=False)[0:5].index.tolist()
temp_df1 = topicSize_df.loc[:, temp_]
temp_df1.plot(marker="v")
plt.title("A_Head topic(top 5) trends")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xticks(rotation=90)
plt.grid()
plt.savefig(DIR_output + "A_Head topic(top 5) trends.png", dpi=300, bbox_inches="tight")
plt.close('all')


# 2. 최근 주요 토픽 5
temp2_ = topicSize_df.iloc[6,:].sort_values(ascending=False).index.tolist()
temp2_ = [i for i in temp2_ if i not in temp_][:5]
temp_df2 = topicSize_df.loc[:, temp2_]
temp_df2.plot(marker="v")
plt.title("B_Recent issue topic(top 5) trends")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xticks(rotation=90)
plt.grid()
plt.savefig(DIR_output + "B_Recent issue topic(top 5) trends.png", dpi=300, bbox_inches="tight")
plt.close('all')

#%%

for i in range(len(output_name_set)):

    DIR_temp = DIR_output + output_name_set[i] + "/"
    dtw_df = pd.read_csv(DIR_temp + output_name_set[i] + ".csv", header=None,  encoding = "utf-8")

    dtw_df.index = dtw_df[0]
    dtw_df = dtw_df.drop([0], axis=1)
    dtw_df = dtw_df.loc[dtw_df[6] >= 0,:] #유의미한 키워드셋
    temp_word_df = dtw_df
    
    # 탑 키워드 추출
    temp_ = dtw_df.sum(axis = 1).sort_values(ascending=False).head(5).index.tolist() 

    # temp_df.high != 0
    temp2_ = [] # 최근 주요 키워드 리스트 10
    temp3_ = [] # 최근 급감 키워드 리스트 10
    
    result_df = pd.DataFrame(columns=["keyword", "grow_ratio", "classification_result"])  # 빈 결과

    for j in range(len(temp_word_df.iloc[:, 0])):
        
        word_name = temp_word_df.index[j]
        # temp_before = 
        temp_word_ = temp_word_df.iloc[j,:].tolist()
        temp_recent = temp_word_[-1]
        temp_before_ = temp_word_[:-2]
        temp_before_ = [i for i in temp_before_ if (i > 0.005) ]
        try : 
            temp_before = np.mean(temp_before_)
        except : 
            temp_before = 0.00001
        
        grow_ratio = (temp_recent / temp_before -1)* 100 
        
        temp_result = [word_name,grow_ratio,"normal"]
        result_df = result_df.append(pd.DataFrame([temp_result], columns=['keyword', "grow_ratio", 'classification_result']), ignore_index=True)
    
    temp2_ = result_df.loc[~result_df["keyword"].isin(temp_) , :].sort_values(by = 'grow_ratio',ascending = False).head(5)['keyword'].tolist()
    temp3_ = result_df.loc[~result_df["keyword"].isin(temp_)  ,:].sort_values(by = 'grow_ratio',ascending = True).head(5)['keyword'].tolist()
    
    
    result_df.loc[result_df["keyword"].isin(temp_),"classification_result"] = 'head'
    result_df.loc[result_df["keyword"].isin(temp2_),"classification_result"] = 'issue'
    result_df.loc[result_df["keyword"].isin(temp3_),"classification_result"] = 'stagnant'
    
    result_df.to_csv(DIR_temp + "keyword_results.csv", index=False, encoding="utf-8")
    
    dtw_df = dtw_df.transpose()
    temp_xtick = list(set(df_after.period))
    temp_xtick.sort()
    dtw_df.index = temp_xtick

    try:
        dtw_df[temp_].plot(marker=">")
        plt.title("A.Head keyword trends for " + output_name_set[i])
        plt.xlabel("Period")
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.xticks(rotation=90)
        plt.grid(True)
        plt.savefig(DIR_temp + "A.Head keyword trends for " + output_name_set[i] + ".png", bbox_inches="tight", dpi=300)
        plt.close('all')
    except:
        pass
    try:
        dtw_df[temp2_].plot(marker=">")
        plt.title("B.Issue keyword trends for " + output_name_set[i])
        plt.xlabel("Period")
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.xticks(rotation=90)
        plt.grid(True)
        plt.savefig(DIR_temp + "B.Issue keyword trends for " + output_name_set[i] + ".png",
                    bbox_inches="tight", dpi=300)
        plt.close('all')
    except:
        pass
    try:
        dtw_df[temp3_].plot(marker=">")
        plt.title("C.Stagnant keyword trends for " + output_name_set[i])
        plt.xlabel("Period")
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.xticks(rotation=90)
        plt.grid(True)
        plt.savefig(DIR_temp + "C.Stagnant keyword trends for " + output_name_set[i] + ".png",
                    bbox_inches="tight", dpi=300)
        plt.close('all')
    except:
        pass

##### 02. dtm network analysis

In [ ]:
node_df = topicSize_df.sum()
node_df = node_df.reset_index()
node_df.columns = ['Id','size']

node_df["category"] = ['application','application','application','technology',
                         'market','application','technology','application',
                         'market','market','technology','market',
                         'application','application','application','technology',
                         'application','application','technology','technology']

node_df.to_csv(DIR_output + "topic_node.csv", index=False)
#%%

select = [x for x in topic_df.columns if x != "period"]

import pandas as pd
import numpy as np
from scipy import stats

col = ['source','target','year','weight','state']
result_df = pd.DataFrame(columns = col)
result_df['state'] = result_df['state'].astype('string')
result_df['year'] = result_df['year'].astype('int')
temp_df = pd.DataFrame(columns = col)
# temp_df = temp_df.astype('string')

temp_xtick = sorted(list(set(topic_df['period'])))
temp_xtick = [int(i) for i in temp_xtick]

for i in range(len(temp_xtick)) :
    
    period = temp_xtick[i]
    
    df_corr = pd.DataFrame() # Correlation matrix
    df_p = pd.DataFrame()  # Matrix of p-values
    df = topic_df.loc[topic_df["period"] == period , select]
    
    # p , corr 연산
    for x in df.columns:
        for y in df.columns:
            corr = stats.kendalltau(df[x], df[y])
            df_corr.loc[x,y] = corr[0]
            df_p.loc[x,y] = corr[1]
    
    # p 기준 투입 
    col_names = df_p.columns.values
    
    for col, row in ((df_p <= 0.05) & (df_p > 0)).iteritems():
        
        topic_1 = col
        
        for topic_2 in col_names[row.values] :
            
            corr_value = df_corr.loc[topic_1,topic_2]
            
            if ((corr_value < 0.5) | (corr_value > 0.9)) : 
                continue
            else :
                topic_1_size = float(topicSize_df.loc[period,topic_1])
                topic_2_size = float(topicSize_df.loc[period,topic_2])
                
                if topic_1_size >= topic_2_size :
                    source = topic_1
                    target = topic_2
                else : 
                    source = topic_2
                    target = topic_1
                
                temp1 = node_df.loc[node_df["Id"] == source,"category"].values[0]
                temp2 = node_df.loc[node_df["Id"] == target,"category"].values[0]
                
                if temp1 == temp2 : state = "inner"
                else : state= "outer"
                    
                temp_df = temp_df.append(pd.DataFrame({'source' : source,
                                                       'target' : target,
                                                       'year' : period,
                                                       'weight' : 1.0,
                                                       'state' : state},index = [0]), ignore_index = True)
                
    # 뉴 소스 및 타겟정보 추가
    # temp_df = temp_df.astype('string')
    
    temp_df = temp_df.drop_duplicates(["source","target","year"]) # 중복제거
    temp_df = temp_df.groupby(["source","target",'state']).sum().reset_index()
    temp_df["year"] = period
    # temp_df[""] =
    
    
    temp_before_df = result_df.loc[result_df["year"] == (period -1), :]
    # temp_df.loc[temp_df[""],'state'] = 
    
    # temp_df['state'] = temp
    # temp_df = temp_df.astype('string')
    
    result_df = pd.concat([result_df,temp_df], ignore_index=True)
    # result_df['state'] = result_df['state'].astype('string')
    
    
#%%

# result_df = result_df.drop_duplicates("corr")
result_df.to_csv(DIR_output + "topic_edge.csv", index=False)

#%%
print(result_df.info())